# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

## Basic FABRIC Slice Configuration

In [1]:
import json
import traceback
# from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Address,Location,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,RAM Available,RAM Capacity,RAM Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated
STAR,"710 North Lake Shore Dr,Chicago, IL 60611","(41.89537135, -87.61663220067463)",6,12,384,384,0,3072,3072,0,121200,121200,0,762,762,0,2,2,0,6,6,0,20,20,0,6,6,0,6,6,0
MASS,"100 Bigelow Street,Holyoke MA 01040","(42.202493000000004, -72.60787662257826)",3,6,192,192,0,1536,1536,0,60600,60600,0,381,381,0,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
SALT,"572 Delong St,Salt Lake City, UT 84104","(40.75707505789612, -111.95346637770317)",3,6,120,192,72,1378,1536,158,60240,60600,360,355,381,26,1,2,1,2,2,0,10,10,0,2,2,0,3,3,0
NCSA,"1725 S Oak St.,Champaign, IL 61820","(40.1035624, -88.2415105)",3,6,168,192,24,1504,1536,32,60500,60600,100,378,381,3,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
UTAH,"875 South West Temple,Salt Lake City, UT 84101","(40.7626391, -111.8939563)",5,10,272,320,48,2496,2560,64,116200,116400,200,633,635,2,2,2,0,4,4,0,16,16,0,4,4,0,5,5,0
WASH,"The Bexley, 1761 Old Meadow Road, McLean, VA 22102, United States of America","(38.91930235, -77.21183383681088)",3,6,62,192,130,1344,1536,192,60110,60600,490,367,381,14,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
DALL,"1950 N Stemmons Fwy,Dallas, TX 75207","(32.8002714, -96.8198113)",3,6,158,192,34,1400,1536,136,60410,60600,190,369,381,12,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
MAX,,"(0, 0)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
MICH,"2530 Draper Dr,Ann Arbor, MI 48109","(42.2931086, -83.7101319)",3,6,132,192,60,1296,1536,240,59040,60600,1560,358,381,23,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
TACC,"10100 Burnet Rd,Austin, TX 78758","(30.3899405, -97.7261806879021)",5,10,264,320,56,2272,2560,288,114500,116400,1900,612,635,23,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0


## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'CNERT 2'

# [site1,site2,site3] = fablib.get_random_sites(count=3)
site1 = site2 = site3 = 'TACC'
print(f"Sites: {site1},{site2},{site3}")


# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h11_name = "h11"
h2_name = "h2"
h22_name = "h22"
h3_name = "h3"

h1_subnet=IPv4Network('10.0.0.0/16')
h1_addr=IPv4Address('10.0.1.1')
h11_addr=IPv4Address('10.0.1.11')

h2_subnet=h1_subnet #IPv4Network('10.0.0.0/16')
h2_addr=IPv4Address('10.0.2.2')
h22_addr=IPv4Address('10.0.2.22')

h3_addr=IPv4Address('10.0.3.3')

host_cores = 2
host_ram = 8
host_disk = 20

net_h1_name = 'net_h1'
net_h11_name = 'net_h11'
net_h2_name = 'net_h2'
net_h22_name = 'net_h22'
net_h3_name = 'net_h3'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

s1_iface_h1,s1_iface_h11,s1_iface_to_s2,s1_iface_to_s3 = "s1_iface_h1","s1_iface_h11","s1_iface_to_s2","s1_iface_to_s3"  
s2_iface_h2,s2_iface_h22,s2_iface_to_s1,s2_iface_to_s3 = "s2_iface_h2","s2_iface_h22","s2_iface_to_s1","s2_iface_to_s3" 
s3_iface_h3, s3_iface_to_s1,s3_iface_to_s2 = "s3_iface_h3", "s3_iface_to_s1","s3_iface_to_s2"

# All node properties
image = 'default_ubuntu_20'

Sites: TACC,TACC,TACC


### Create the Slice

In [4]:
import datetime

try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    def addSwitchNIC(nodeName,site,ifaceNames):
        node = slice.add_node(name=nodeName, site=site,  image=image, cores=switch_cores, ram=switch_ram, disk=switch_disk)
        ifaceObj = {}
        for name in ifaceNames:
            iface =  node.add_component(model='NIC_Basic', name=name).get_interfaces()[0]
            ifaceObj[name] = iface
        return node, ifaceObj
    
    # Add switch node s1
    s1, s1_ifaces = addSwitchNIC(s1_name,site1,[s1_iface_h1,s1_iface_h11,s1_iface_to_s2,s1_iface_to_s3])

    # Add switch node s2
    s2, s2_ifaces = addSwitchNIC(s2_name,site2,[s2_iface_h2,s2_iface_h22,s2_iface_to_s1,s2_iface_to_s3])
    
    # Add switch node s3
    s3, s3_ifaces = addSwitchNIC(s3_name,site3,[s3_iface_h3, s3_iface_to_s1,s3_iface_to_s2])
    
    def addHostNIC(nodeName,componentName,site):
        node = slice.add_node(name=nodeName, site=site, image=image, cores=host_cores, ram=host_ram, disk=host_disk)
        iface = node.add_component(model='NIC_Basic', name=componentName).get_interfaces()[0]
        return node,iface
    
    # Add host node h1
    h1,h1_iface = addHostNIC(h1_name,"h1_nic",site1)

    # Add host node h11
    h11 , h11_iface = addHostNIC(h11_name,"h11_nic",site1)
    
    # Add host node h2
    h2, h2_iface = addHostNIC(h2_name,"h2_nic",site2)
    
    # Add host node h22
    h22, h22_iface = addHostNIC(h22_name,"h22_nic",site2)
    
    # Add host node h3
    h3, h3_iface = addHostNIC(h3_name,"h3_nic",site2)
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_ifaces[s1_iface_to_s2], s2_ifaces[s2_iface_to_s1]])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_ifaces[s2_iface_to_s3], s3_ifaces[s3_iface_to_s2]])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_ifaces[s3_iface_to_s1], s1_ifaces[s1_iface_to_s3]])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_ifaces[s1_iface_h1], h1_iface])
    host_net11 = slice.add_l2network(name=net_h11_name, interfaces=[s1_ifaces[s1_iface_h11], h11_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_ifaces[s2_iface_h2], h2_iface])
    host_net22 = slice.add_l2network(name=net_h22_name, interfaces=[s2_ifaces[s2_iface_h22], h22_iface])
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_ifaces[s3_iface_h3], h3_iface])
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


Retry: 3, Time: 341 sec


ID,c8af453e-4f25-416c-b5de-c1ef69309336
Name,CNERT 2
Lease Expiration (UTC),2023-01-15 18:59:57 +0000
Lease Start (UTC),2023-01-14 18:59:59 +0000
Project ID,f8a6e0b0-ad14-47cb-9764-74c20ef3e4fc
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
c894d690-fd7d-4c00-993e-a1effc655796,s1,2,8,100,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.79,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.79,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
e5b1a58f-0077-43df-88f7-81458f246436,s2,2,8,100,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.120,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.120,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
13c4a558-37f1-4925-b88f-862ef357283f,s3,2,8,100,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.102,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.102,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
2ab0ce1e-4b9f-455f-b585-dea061c7ceb2,h1,2,8,100,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.85,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.85,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
f9bd4fcb-847f-4d71-869f-f74709c54181,h11,2,8,100,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.110,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.110,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
b6584c26-1df4-4cac-8c8b-6f0a68aa4b02,h2,2,8,100,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.93,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.93,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
e5198885-2b16-49e7-b2ef-519b67033b41,h22,2,8,100,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.106,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.106,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key
0344e611-0465-4045-b801-69e288cd5a9f,h3,2,8,100,default_ubuntu_20,qcow2,tacc-w2.fabric-testbed.net,TACC,ubuntu,129.114.110.74,Active,,ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.74,/home/fabric/work/fabric_config/.ssh/slice_key.pub,/home/fabric/work/fabric_config/.ssh/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
f7498958-8e99-4cc1-a3af-a6a232ca2296,net_s1_s2,L2,L2Bridge,TACC,None,None,Active,
c6029d90-ac63-4a77-8a06-5d174fc7aef4,net_s2_s3,L2,L2Bridge,TACC,None,None,Active,
3fbaaded-2c6c-4c60-8c04-7248017f772c,net_s1_s3,L2,L2Bridge,TACC,None,None,Active,
28b91522-23ab-422c-89e7-753a7b75583a,net_h1,L2,L2Bridge,TACC,None,None,Active,
1eced5b6-25b2-4550-bad2-9e39835df764,net_h11,L2,L2Bridge,TACC,None,None,Active,
b73d94aa-b984-4926-ac59-121a672f4f2f,net_h2,L2,L2Bridge,TACC,None,None,Active,
ecac1d88-6379-47f6-a170-56177e7d43e9,net_h22,L2,L2Bridge,TACC,None,None,Active,
e97d7dd6-92fc-4180-975d-99576353fa14,net_h3,L2,L2Bridge,TACC,None,None,Active,



Time to stable 341 seconds
Running post_boot_config ...  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
  sudo: nmcli: command not found
 Time to post boot config 397 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
s1-s1_iface_h1-p1,s1,net_h1,100,None,2A:B7:3C:BF:5B:66,ens10,ens10
s1-s1_iface_to_s3-p1,s1,net_s1_s3,100,None,26:E1:33:6D:4F:8B,ens8,ens8
s1-s1_iface_h11-p1,s1,net_h11,100,None,26:C6:B9:7D:D5:22,ens7,ens7
s1-s1_iface_to_s2-p1,s1,net_s1_s2,100,None,2A:95:DD:38:25:EF,ens9,ens9
s2-s2_iface_h22-p1,s2,net_h22,100,None,2E:7A:CC:99:81:29,ens7,ens7
s2-s2_iface_to_s1-p1,s2,net_s1_s2,100,None,32:35:D1:70:7E:1D,ens8,ens8
s2-s2_iface_h2-p1,s2,net_h2,100,None,3A:AC:16:A5:F5:44,ens10,ens10
s2-s2_iface_to_s3-p1,s2,net_s2_s3,100,None,32:7C:C3:67:7A:6C,ens9,ens9
s3-s3_iface_to_s2-p1,s3,net_s2_s3,100,None,3E:02:A7:B5:E1:72,ens7,ens7
s3-s3_iface_h3-p1,s3,net_h3,100,None,3E:22:DC:D7:82:DE,ens8,ens8



Time to print interfaces 429 seconds


In [5]:
def checkInetConnection(node):
    try:

        #If the node is an IPv6 Node then configure NAT64
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            print("This code only works with ip4, reconfigure nodes")
            return

        #Access non-IPv6 Services
        stdout, stderr = node.execute(f'sudo apt-get update -qq && sudo apt install -y net-tools && git clone https://github.com/fabric-testbed/jupyter-examples.git && sudo apt install -y ffmpeg')
        # print(stdout)
        # print(stderr)

        stdout, stderr = node.execute(f'ls jupyter-examples')
        print(stdout)
        print(stderr)
        
        stdout, stderr = node.execute(f'ifconfig | grep ens')
        print(stdout)
        print(stderr)


    except Exception as e:
        print(f"Exception: {e}")

In [6]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        checkInetConnection(node)
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
Reading package lists... 

 
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  net-tools
0 upgraded, 1 newly installed, 0 to remove and 183 not upgraded.
Need to get 196 kB of archives.
After this operation, 864 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal/main amd64 net-tools amd64 1.60+git20180626.aebd88e-1ubuntu1 [196 kB]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
  debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
  dpkg-preconfigure: unable to re-open stdin: 
 Fetched 196 kB in 1s (253 kB/s)
Selecting previously unselected package net-tools.
(Reading database ... 63577 files and dire

### Print Details

In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("----")
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        ssh = node.get_ssh_command().replace("ssh -i /home/fabric/work/fabric_config/.ssh/id_rsa -J durbek_gafurov_0000000854@bastion-1.fabric-testbed.net","./connect.sh")
        print(f"   SSH Command       : {ssh}")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Model            : {component.get_model()}")
        print(f"   Interfaces        :  ", end = " ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           MAC                 : {interface.get_mac()}") 
            print(interface.get_os_interface(), end =" ")
            # stdout, stderr = node.execute(f'sudo apt install -y net-tools &&  sudo ifconfig {interface.get_os_interface()} up')
    for network in slice.get_l2networks():
        print("Network:")
        print(f"    Name:            {network.get_name()}")
    # print(f"Interface Map: {slice.get_interface_map()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

----
Node:
   Name              : s1
   Cores             : 2
   RAM               : 8
   Disk              : 100
   Reservation State : Active
   SSH Command       : ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.79
      Name             : s1-s1_iface_h1
      Model            : NIC_Basic
      Name             : s1-s1_iface_to_s3
      Model            : NIC_Basic
      Name             : s1-s1_iface_h11
      Model            : NIC_Basic
      Name             : s1-s1_iface_to_s2
      Model            : NIC_Basic
   Interfaces        :          Name                : s1-s1_iface_h1-p1
           MAC                 : 2A:B7:3C:BF:5B:66
ens10 Reading package lists... 

 
Building dependency tree...
Reading state information...
net-tools is already the newest version (1.60+git20180626.aebd88e-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 183 not upgraded.
       Name                : s1-s1_iface_to_s3-

In [8]:
try:
    for node in slice.get_nodes():
        print("----")
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            stdout, stderr = node.execute(f'sudo apt install -y net-tools &&  sudo ifconfig {interface.get_os_interface()} up ')
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

----
Node:
   Name              : s1
      Name             : s1-s1_iface_h1
      Name             : s1-s1_iface_to_s3
      Name             : s1-s1_iface_h11
      Name             : s1-s1_iface_to_s2
       Name                : s1-s1_iface_h1-p1
Reading package lists... 

 
Building dependency tree...
Reading state information...
net-tools is already the newest version (1.60+git20180626.aebd88e-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 183 not upgraded.
       Name                : s1-s1_iface_to_s3-p1
Reading package lists... 

 
Building dependency tree...
Reading state information...
net-tools is already the newest version (1.60+git20180626.aebd88e-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 183 not upgraded.
       Name                : s1-s1_iface_h11-p1
Reading package lists... 

 
Building dependency tree...
Reading state information...
net-tools is already the newest version (1.60+git20180626.aebd88e-1ubuntu1).
0 upgraded, 0 newly installed, 0

## Configure Nodes


In [9]:
config_threads = {}

In [10]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/Durbek-Gafur/P4-VCC.git && git clone https://github.com/p4lang/tutorials.git/" 
def addIP(name,net_name,node_addr,node_subnet):
    node = slice.get_node(name=name)        
    h1_os_iface = node.get_interface(network_name=net_name).ip_addr_add(addr=node_addr, subnet=node_subnet)
    config_thread = node.execute_thread(host_config_script)
    config_threads[node]=config_thread
    return node
    
try:
    
    h1 = addIP(h1_name,net_h1_name,h1_addr,h1_subnet)
    # print(h1.execute(f'sudo arp -i ens7 -s 10.0.1.0 08:00:00:00:01:00 && arp -a'))
    h11 = addIP(h11_name,net_h11_name,h11_addr,h1_subnet)
    # print(h11.execute(f'sudo arp -i ens7 -s 10.0.1.10 08:00:00:00:01:00 && arp -a'))
    h2 = addIP(h2_name,net_h2_name,h2_addr,h2_subnet)
    # print(h2.execute(f'sudo arp -i ens7 -s 10.0.2.3 08:00:00:00:02:00 && arp -a'))
    h22 = addIP(h22_name,net_h22_name,h22_addr,h2_subnet)
    # print(h22.execute(f'sudo arp -i ens7 -s 10.0.2.23 08:00:00:00:02:00 && arp -a'))


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [11]:
def checkIP(node):
    try:        
        stdout, stderr = node.execute(f'ifconfig | grep "inet 10.0."')
        print(stdout)
        print(stderr)
    except Exception as e:
        print(f"Exception: {e}")

In [12]:
try: 
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        checkIP(node)
        # print(f"stdout: {stdout}")
        # print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 
config_threads = {}

Config thread node h1 complete
        inet 10.0.1.1  netmask 255.255.0.0  broadcast 0.0.0.0
        inet 10.0.1.1  netmask 255.255.0.0  broadcast 0.0.0.0


Config thread node h11 complete
        inet 10.0.1.11  netmask 255.255.0.0  broadcast 0.0.0.0
        inet 10.0.1.11  netmask 255.255.0.0  broadcast 0.0.0.0


Config thread node h2 complete
        inet 10.0.2.2  netmask 255.255.0.0  broadcast 0.0.0.0
        inet 10.0.2.2  netmask 255.255.0.0  broadcast 0.0.0.0


Config thread node h22 complete
        inet 10.0.2.22  netmask 255.255.0.0  broadcast 0.0.0.0
        inet 10.0.2.22  netmask 255.255.0.0  broadcast 0.0.0.0




## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [13]:
s1 = slice.get_node(name=s1_name)
s2 = slice.get_node(name=s2_name)
s3 = slice.get_node(name=s3_name)

In [14]:
print(s1)

-----------------  -------------------------------------------------------------------------------------------------------------------------
ID                 c894d690-fd7d-4c00-993e-a1effc655796
Name               s1
Cores              2
RAM                8
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               tacc-w2.fabric-testbed.net
Site               TACC
Management IP      129.114.110.79
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/.ssh/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.79
-----------------  -------------------------------------------------------------------------------------------------------------------------


In [15]:
h1Mac = h1.get_interface(network_name=net_h1_name).get_mac()
h11Mac = h11.get_interface(network_name=net_h11_name).get_mac()
h2Mac = h2.get_interface(network_name=net_h2_name).get_mac()
h22Mac = h22.get_interface(network_name=net_h22_name).get_mac()

s1_net_s1_s2_Mac = s1.get_interface(network_name=net_s1_s2_name).get_mac()
s2_net_s1_s2_Mac = s2.get_interface(network_name=net_s1_s2_name).get_mac()

s1_net_s1_s3_Mac = s1.get_interface(network_name=net_s1_s3_name).get_mac()
s3_net_s1_s3_Mac = s3.get_interface(network_name=net_s1_s3_name).get_mac()

s2_net_s2_s3_Mac = s2.get_interface(network_name=net_s2_s3_name).get_mac()
s3_net_s2_s3_Mac = s3.get_interface(network_name=net_s2_s3_name).get_mac()

print(h1Mac,h11Mac,h2Mac,h22Mac)
def printToFile(text,filename):
    text_file = open(filename, "w")
    n = text_file.write(text)
    text_file.close()
s1_command = """table_clear MyIngress.ipv4_lpm 
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.11/32 => {} 2
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.2/32 => {} 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.22/32 => {} 4
""".format(h1Mac, h11Mac,s2_net_s1_s2_Mac,s3_net_s1_s3_Mac)
s2_command = """table_clear MyIngress.ipv4_lpm 
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => {} 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.11/32 => {} 4
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.2/32 => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.22/32 => {} 2
""".format(s1_net_s1_s2_Mac,s3_net_s2_s3_Mac,h2Mac,h22Mac)
s3_command = """table_clear MyIngress.ipv4_lpm 
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.0/24 => {} 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.0/24 => {} 2
""".format(s1_net_s1_s3_Mac,s2_net_s2_s3_Mac)
printToFile(s1_command, "./scripts/s1command.txt")
printToFile(s2_command, "./scripts/s2command.txt")
printToFile(s3_command, "./scripts/s3command.txt")
print(s1_command, "./scripts/s1command.txt")
print(s2_command, "./scripts/s2command.txt")
print(s3_command, "./scripts/s3command.txt")

3E:98:DB:AC:A0:F9 46:5B:BC:9B:23:72 4A:36:9F:73:88:BF 4A:94:51:98:CB:BE
table_clear MyIngress.ipv4_lpm 
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => 3E:98:DB:AC:A0:F9 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.11/32 => 46:5B:BC:9B:23:72 2
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.2/32 => 32:35:D1:70:7E:1D 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.22/32 => 3E:5A:26:50:40:02 4
 ./scripts/s1command.txt
table_clear MyIngress.ipv4_lpm 
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.1/32 => 2A:95:DD:38:25:EF 3
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.11/32 => 3E:02:A7:B5:E1:72 4
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.2/32 => 4A:36:9F:73:88:BF 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.22/32 => 4A:94:51:98:CB:BE 2
 ./scripts/s2command.txt
table_clear MyIngress.ipv4_lpm 
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.1.0/24 => 26:E1:33:6D:4F:8B 1
table_add MyIngress.ipv4_lpm ipv4_forward 10.0.2.0/24 => 32:7C:C3:67:7A:6C 2
 ./scrip

In [16]:
s1_net_s1_h1_Mac = s1.get_interface(network_name=net_h1_name).get_mac()
print(s1.get_interface(network_name=net_h1_name))
s1_net_s1_h11_Mac = s1.get_interface(network_name=net_h11_name).get_mac()
print(s1.get_interface(network_name=net_h11_name))
h1_arpcommand = """sudo arp -i ens7 -s 10.0.2.2 {} && arp -a""".format(s1_net_s1_h1_Mac)
h11_arpcommand = """sudo arp -i ens7 -s 10.0.2.22 {} && arp -a""".format(s1_net_s1_h11_Mac)
print(h1_arpcommand, "./scripts/s1_arpcommand.txt")
print(h11_arpcommand, "./scripts/s2_arpcommand.txt")
stdout, stderr = h1.execute(h1_arpcommand)
print(stdout)
print(stderr)
stdout, stderr = h11.execute(h11_arpcommand)
print(stdout)
print(stderr)
# printToFile(h1_arpcommand, "./scripts/s1_arpcommand.txt")
# printToFile(h11_arpcommand, "./scripts/s2_arpcommand.txt")
# print(h1_arpcommand, "./scripts/s1_arpcommand.txt")
# print(h11_arpcommand, "./scripts/s2_arpcommand.txt")

---------------  -----------------
Name             s1-s1_iface_h1-p1
Network          net_h1
Bandwidth        100
VLAN
MAC              2A:B7:3C:BF:5B:66
Physical Device  ens10
Device           ens10
---------------  -----------------
---------------  ------------------
Name             s1-s1_iface_h11-p1
Network          net_h11
Bandwidth        100
VLAN
MAC              26:C6:B9:7D:D5:22
Physical Device  ens7
Device           ens7
---------------  ------------------
sudo arp -i ens7 -s 10.0.2.2 2A:B7:3C:BF:5B:66 && arp -a ./scripts/s1_arpcommand.txt
sudo arp -i ens7 -s 10.0.2.22 26:C6:B9:7D:D5:22 && arp -a ./scripts/s2_arpcommand.txt
? (10.20.4.11) at fa:16:3e:cc:48:d2 [ether] on ens3
_gateway (10.20.4.1) at fa:16:3e:bc:de:76 [ether] on ens3
? (10.0.2.2) at 2a:b7:3c:bf:5b:66 [ether] PERM on ens7
? (10.20.4.11) at fa:16:3e:cc:48:d2 [ether] on ens3
_gateway (10.20.4.1) at fa:16:3e:bc:de:76 [ether] on ens3
? (10.0.2.2) at 2a:b7:3c:bf:5b:66 [ether] PERM on ens7


? (10.0.2.22) at 26:c6:

In [17]:
def configureSwitch(name,network_names):
    try:
        node = slice.get_node(name=name)
        os_ifaces = ""
        for net_name in network_names:
            iface = node.get_interface(network_name=net_name).get_os_interface()
            if iface is None:
                print("iface is none")
                return
            os_ifaces=os_ifaces+iface+" "

        file_attributes = node.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
        command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {os_ifaces}  > /tmp/script.log 2>&1' && tail /tmp/script.log"
        config_threads[node] = node.execute_thread(command)
        print(command)


    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()

In [18]:
configureSwitch(s1_name,[net_h1_name,net_h11_name,net_s1_s2_name,net_s1_s3_name]) 
configureSwitch(s2_name,[net_h2_name,net_h22_name,net_s1_s2_name,net_s2_s3_name])
configureSwitch(s3_name,[net_s1_s3_name,net_s2_s3_name])

chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens10 ens7 ens9 ens8   > /tmp/script.log 2>&1' && tail /tmp/script.log
chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens10 ens7 ens8 ens9   > /tmp/script.log 2>&1' && tail /tmp/script.log
chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh ens9 ens7   > /tmp/script.log 2>&1' && tail /tmp/script.log


In [19]:
try: 
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 


Config thread node s1 complete
stdout: V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
simple_switch --interface 1@ens10 --interface 2@ens7 --interface 3@ens9 --interface 4@ens8 /root/tutorials/exercises/basic_tunnel/basic_tunnel.json
done!

stderr: 
Config thread node s2 complete
stdout: V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
simple_switch --interface 1@ens10 --interface 2@ens7 --interface 3@ens8 --interface 4@ens9 /root/tutorials/exercises/basic_tunnel/basic_tunnel.json
done!

stderr: 
Config thread node s3 complete
stdout: V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
simple_switch --interface 1@ens9 --interface 2@ens7 /root/tutorials/exercises/basic_tunnel/basic_tunnel.json
done!

stderr: 


### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [20]:
s1

In [21]:

for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}command.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))
    stdout = switch_node.execute("echo 'sudo docker exec -it fabric_p4 /bin/bash' > dc.sh && sudo chmod +x dc.sh && ls -lh")
    print("stdout: {}".format(stdout))

Swtitch Name        : s1
Management IP    : 129.114.110.79
s1command.txt
file_attributes: -rw-rw-r--   1 1000     1000          342 14 Jan 19:20 ?
Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-0a:00:01:01/32
action:              ipv4_forward
runtime data:        3e:98:db:ac:a0:f9	00:01
Entry has been added with handle 0
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-0a:00:01:0b/32
action:              ipv4_forward
runtime data:        46:5b:bc:9b:23:72	00:02
Entry has been added with handle 1
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-0a:00:02:02/32
action:              ipv4_forward
runtime data:        32:35:d1:70:7e:1d	00:03
Entry has been added with handle 2
RuntimeCmd: Adding entry to lpm match table MyIngress.ipv4_lpm
match key:           LPM-0a:00:02:16/32
ac

## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [22]:
h1.upload_file('scripts/send.py', 'P4-VCC/exercises/basic_tunnel/send_modified.py')
h1.upload_file('scripts/receive.py', 'P4-VCC/exercises/basic_tunnel/receive_modified.py')
h2.upload_file('scripts/send.py', 'P4-VCC/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'P4-VCC/exercises/basic_tunnel/receive_modified.py')
h11.upload_file('scripts/send.py', 'P4-VCC/exercises/basic_tunnel/send_modified.py')
h11.upload_file('scripts/receive.py', 'P4-VCC/exercises/basic_tunnel/receive_modified.py')
h22.upload_file('scripts/send.py', 'P4-VCC/exercises/basic_tunnel/send_modified.py')
h22.upload_file('scripts/receive.py', 'P4-VCC/exercises/basic_tunnel/receive_modified.py')

<SFTPAttributes: [ size=1137 uid=1000 gid=1000 mode=0o100664 atime=1673724056 mtime=1673724056 ]>

## H1 sends packet, H2 recieves

In [2]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 P4-VCC/exercises/basic/send_modified.py 10.0.2.2 "message100" 50\'')
print(h2.execute('sudo timeout 30 sudo python3 P4-VCC/exercises/basic/receive_modified.py')[0])

NameError: name 'h1' is not defined

In [24]:
h1.execute('sudo python3 P4-VCC/exercises/basic/send.py 10.0.2.22 "message100" 50')

sending on interface ens7 to 10.0.2.22
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 3e:98:db:ac:a0:f9
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 50
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x63af
     src       = 10.0.1.1
     dst       = 10.0.2.22
     \options   \
###[ TCP ]### 
        sport     = 59238
        dport     = 1234
        seq       = 0
        ack       = 0
        dataofs   = 5
        reserved  = 0
        flags     = S
        window    = 8192
        chksum    = 0xb4e7
        urgptr    = 0
        options   = []
###[ Raw ]### 
           load      = 'message100'



("sending on interface ens7 to 10.0.2.22\n###[ Ethernet ]### \n  dst       = ff:ff:ff:ff:ff:ff\n  src       = 3e:98:db:ac:a0:f9\n  type      = IPv4\n###[ IP ]### \n     version   = 4\n     ihl       = 5\n     tos       = 0x0\n     len       = 50\n     id        = 1\n     flags     = \n     frag      = 0\n     ttl       = 64\n     proto     = tcp\n     chksum    = 0x63af\n     src       = 10.0.1.1\n     dst       = 10.0.2.22\n     \\options   \\\n###[ TCP ]### \n        sport     = 59238\n        dport     = 1234\n        seq       = 0\n        ack       = 0\n        dataofs   = 5\n        reserved  = 0\n        flags     = S\n        window    = 8192\n        chksum    = 0xb4e7\n        urgptr    = 0\n        options   = []\n###[ Raw ]### \n           load      = 'message100'\n\n",
 '')

In [25]:
h11.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 P4-VCC/exercises/basic/send.py 10.0.2.22 "message100" 50\'')
print(h22.execute('sudo timeout 30 sudo python3 P4-VCC/exercises/basic/receive.py')[0])

sniffing on ens7
sniffing on ens7



In [26]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 P4-VCC/exercises/basic_tunnel/send_modified.py --dst_id 2 10.0.2.2 "message100"\'')
print(h2.execute('sudo timeout 30 sudo python3 P4-VCC/exercises/basic_tunnel/receive_modified.py')[0])

sniffing on ens7
sniffing on ens7



## H1 sends packet, H11 recieves

In [27]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 P4-VCC/exercises/basic/send.py 10.0.1.11 "message100" 50\'')
print(h11.execute('sudo timeout 30 sudo python3 P4-VCC/exercises/basic/receive.py')[0])

sniffing on ens7
sniffing on ens7



## H2 sends packet, H1 recieves

In [28]:
h2.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 P4-VCC/exercises/basic/send.py 10.0.1.1 "message100" 50\'')
print(h1.execute('sudo timeout 30 sudo python3 P4-VCC/exercises/basic/receive.py')[0])

sniffing on ens7
sniffing on ens7



## H22 sends packet, H11 recieves

In [29]:

h22.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py 10.0.1.11 "message100" 50\'')
print(h11.execute('sudo timeout 30 sudo python3 P4-VCC/exercises/basic/receive.py')[0])

sniffing on ens7
sniffing on ens7



In [30]:

h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 P4-VCC/exercises/basic_tunnel/send_modified.py 10.0.1.11 "message100"\'')
print(h11.execute('sudo timeout 30 sudo python3 P4-VCC/exercises/basic/receive.py')[0])

sniffing on ens7
sniffing on ens7



## Delete Slice

In [31]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Fail: {e}")
#     traceback.print_exc()